In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'medical-insurance-payout:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2241021%2F3750249%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240417%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240417T150030Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D79df3531d7607a6b72cfc8b054fbdda7acf7b61e7eafaa6dda41f85cabf1f84a05b6d57ea70f9e098330366057bf0b6127608cebc6c3b9862be13742b8eeda4a719623626ed0a1f5a1383a4cec453be0df0ddfe8be55aaf33490792585c9466684b3b83bfc5ea923ac199428c93b1896c49029d6f65ec88f713b1ba2168da0b8bc011a50e442e206282b373a9154012ba2074625af2f75a87ce2be18a9bd454e2e13ab72d29b9c20f07d2fdf8c9f0b6bb5350c3aa1ce774e05efe917b7de93330ba94dd92776766db3df06ae7e6144a146dee5797103140629248357894ca88536f6a54c8fb450c90731019e8abe7bb17921531b9dd5e786c2bdc60b61f4bb90'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 16423 bytes downloaded
Downloaded and uncompressed: medical-insurance-payout
Data source import complete.


## Target

ACME Insurance Inc. provides affordable health insurance to thousands of customers across the United States. They want to launch an attractive customer acquisition campaign. Your task is to study the data provided by the company, such as age, gender, BMI, children, smoking habits and region of residence of current customers and propose possible strategies.

## Introduction

Analysis of available data looking for possible trend lines to estimate the annual medical expenditure of new and existing customers, establishing the reduced annual premium price as the main attraction, while the company promotes healthy habits among its customers by reducing the risk of unpredictable extra expenses.

## Method

-Data analysis and visualisations: Python and R  
-Model: Estimating annual medical expenditure using available information (Python, lazypredict, skalear)

In [1]:
%pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 2.4 MB/s eta 0:00:00


In [3]:
%pip install lazypredict

In [4]:
# tratamiento de los datos
# ============================================
import pandas as pd
import numpy as np
from scipy.stats import skew
import scipy.stats as st
import sidetable
from scipy.stats import chi2_contingency
from scipy import stats
from scipy.stats import pearsonr
import statsmodels.api as sm
import pingouin as pg
from scipy.stats import kurtosistest
import statsmodels.api as sm
from lazypredict.Supervised import LazyRegressor

# librerías para la visualización de los datos
# ============================================
import matplotlib.pyplot as plt
import seaborn as sns

# Establecer tamaño fijo de gráficas
# ==================================
plt.rcParams["figure.figsize"] = (10,8)

# Configuración warnings
# ======================
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'sidetable'

In [ ]:
# Objectives
#Description of data, identification of variables to be taken into account for the calculation of health insurance premiums.

#Predict amount of charges

In [ ]:
df = pd.read_csv("/kaggle/input/medical-insurance-payout/expenses.csv", index_col = 0).reset_index()
df.head(2)

In [ ]:
## Understand the variables we have and what our dataframe looks like.

In [ ]:
# number of rows and columns in the dataframe

df.shape

In [ ]:
# general data frame information

df.info()

In [ ]:
df.isnull().sum()


In [ ]:
# duplicates

df.duplicated().sum()

In [ ]:
df[df.duplicated()== True]

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.to_csv('insurance_ok.csv')

In [ ]:
### Statistics

In [ ]:
# leading statistics of the numeric columns.

df.describe(include='all')

Data info:
- Smoking habits, majority non-smokers (274 smokers)
- BMI mean 30, overweight (more than 75% over BMI 25)

In [ ]:
# Outliers

In [ ]:
numericas = df.select_dtypes(include=np.number)
numericas

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(20,4))
axes = axes.flat
for indice, columna in enumerate(numericas.columns):
    sns.boxplot(x = numericas[columna], data = df, ax=axes[indice], color = "turquoise")
plt.tight_layout()
plt.show();

In [ ]:
# I'm going to look at the distribution of variable charges
sns.set(style="ticks")
sns.set_style("darkgrid")
sns.distplot(
    df["charges"],
    color = "blue",
    kde_kws = {"shade": True, "linewidth": 1});


In [ ]:
### Not normal

In [ ]:
## We analyse the numerical variables of the dataset

In [ ]:
### Distributions

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (30, 10))
axes = axes.flat#iterator
for i, colum in enumerate(numericas.columns):
    sns.histplot(
        data = numericas[colum],
        kde = True,
        color = "purple",
        line_kws = {"linewidth": 2},
        alpha = 0.5,
        ax = axes[i])
    axes[i].set_title(colum, fontsize = 20, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")
fig.tight_layout();

### Relationship to the response variable

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (30, 10))
axes = axes.flat
lista_colores = ["cyan", "magenta", "orange"]

columnas_numeric = df.select_dtypes(include = np.number).columns
columnas_numeric = columnas_numeric.drop("charges")

for i, colum in enumerate(columnas_numeric):
    sns.regplot(
        x = df[colum],
        y = df["charges"],
        color = lista_colores[i],
        marker = ".",
        scatter_kws = {"alpha": 0.4},
        line_kws = {"color": "black", "alpha": 0.7 },
        ax = axes[i])

    axes[i].set_title(f"Charges vs {colum}", fontsize = 20, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

fig.tight_layout();


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,7))
categoricas=df.select_dtypes(include='object')
for i in range(len(categoricas.columns)):
    sns.scatterplot(x='age', y='charges', data = df,
                    s = 25,
                    hue = categoricas.columns[i],
                    ax=axes[i])
plt.show();

Here we see that in age, there are about three clear trend lines in the distribution of our data.  
**-------------------------------------All 3 increase with age and tobacco-------------------------------------**

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,7))

for i in range(len(categoricas.columns)):
    sns.scatterplot(x='bmi', y='charges', data = df,
                    s = 25,
                    hue = categoricas.columns[i],
                    ax=axes[i])
plt.show();

**-------------------------------We see that there is a clear increase  in charges with tobacco--------------------------**

In [ ]:
mask = np.triu(np.ones_like(df.corr(), dtype = np.bool))
sns.heatmap(df.corr(method='spearman'),
           cmap = "icefire",
            mask = mask,
           annot = True);

## Categorical variables

### I create some graphs to go deeper into the data

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20, 15))
axes = axes.flat
lista_colores = [ "magenta", "orange","blue", "green"]
columnas = df.select_dtypes(include = 'object')
columnas = columnas.columns
for i, colum in enumerate(columnas):
    sns.violinplot(
        data = df,
        x = colum,
        y = 'charges',
        color = lista_colores[i],
        line_kws = {"color": "black", "alpha": 0.7 },
        ax = axes[i])
    axes[i].set_title(colum, fontsize = 15, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")

fig.tight_layout();

In [ ]:
sns.barplot(x='region', y='charges', data=df, palette='Spectral')
plt.ylim(0,40000);

In [ ]:
sns.barplot(x='children', y='charges', data=df, palette='Set2')
plt.ylim(0,40000);

In [ ]:
sns.barplot(x='smoker', y='charges', data=df, palette='Spectral')
plt.ylim(0,40000);

In [ ]:
sns.barplot(x='sex', y='charges' ,data=df, palette='Spectral')
plt.ylim(0,40000);

modelo

In [ ]:
df['sex'] = df['sex'].map({'female':0,'male':1})
df['smoker'] = df['smoker'].map({'no':0,'yes':1})
df['region'] = df['region'].map({'northeast':1,'northwest':2,'southeast':3,'southwest':4})
df.head()

In [ ]:
df.info()

In [ ]:
X = df.drop('charges',axis=1)
y = df['charges']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from lazypredict.Supervised import LazyRegressor

In [ ]:
clf = LazyRegressor(verbose=0)
models,predictions = clf.fit(x_train, x_test, y_train, y_test)
models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA

In [ ]:
# iniciamos la regresión lineal. n_jobs hace referencia al número de nucleos que usaremos de nuestro ordenador. Al indicar -1 significa
#  que los usaremos todos.

lr = LinearRegression(n_jobs=-1)

In [ ]:
# fiteamos el modelo, lo que significa que le pasamos los datos de entrenamiento para que aprenda el algoritmo. Fijaros que para que aprenda,
#  les paso solo los datos de entrenamiento

lr.fit(x_train, y_train)

In [ ]:
# es el momento de hacer las predicciones, para eso usarmos el método predict() de sklearn

y_predict_train = lr.predict(x_train) # hacemos las predicciones para las casas que tenemos en el los datos de entrenamiento
y_predict_test = lr.predict(x_test) # hacemos las predicciones para las casas que tenemos en los datos de test

In [ ]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_predict_train, 'Set': ['Train']*len(y_train)})
test_df  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)
resultados.head()

In [ ]:
resultados['residuos'] = resultados['Real'] - resultados['Predicted']
resultados.head()

In [ ]:
test_df  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test),'MAE': mean_absolute_error(y_test, y_predict_test),'MSE': mean_squared_error(y_test, y_predict_test),'RMSE': np.sqrt(mean_squared_error(y_test, y_predict_test)),'R2':  r2_score(y_test, y_predict_test),"modelo": "LinearRegression"})

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(20,20))


# ploteamos los reales vs los predichos
sns.regplot(data = resultados[resultados['Set'] == "Train"],
            x = "Real",
            y = "Predicted",
            ax = ax[0,0],
            color = "grey",
            line_kws = {"color": "red", "alpha": 0.7 })


sns.regplot(data = resultados[resultados['Set'] == "Test"],
            x = "Real",
            y = "Predicted",
            color = "gray",
            line_kws = {"color": "red", "alpha": 0.7 },
            ax = ax[1,0])


# ploteamos los residuos
sns.histplot(resultados[resultados['Set'] == "Train"],
             x="residuos",
             color ="grey",
             kde=True,
             ax = ax[0,1])


sns.histplot(resultados[resultados['Set'] == "Test"],
             x="residuos",
             color = "grey",
             kde=True,
             ax = ax[1,1])

ax[0,0].set_title("Train reales vs predichos", fontsize = 15, fontweight = "bold")
ax[0,1].set_title("Train residuos", fontsize = 15, fontweight = "bold")
ax[1,0].set_title("Test reales vs predichos", fontsize = 15, fontweight = "bold")
ax[1,1].set_title("Test residuos", fontsize = 15, fontweight = "bold");


In [ ]:
resultados_metricas = {'MAE': [mean_absolute_error(y_test, y_predict_test), mean_absolute_error(y_train, y_predict_train)],
                'MSE': [mean_squared_error(y_test, y_predict_test), mean_squared_error(y_train, y_predict_train)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_predict_test)), np.sqrt(mean_squared_error(y_train, y_predict_train))],
                'R2':  [r2_score(y_test, y_predict_test), r2_score(y_train, y_predict_train)],
                 "set": ["test", "train"],
                 "modelo": ["Linear Regresion", "LinearRegression"]}

df_resultados = pd.DataFrame(resultados_metricas)

df_resultados

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Standardize the dataset
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

In [ ]:
# Hyperparameters for GradientBoostingRegressor
#
gbr_params = {'n_estimators': 1000,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'absolute_error'}

In [ ]:
# Create an instance of gradient boosting regressor
#
gbr = GradientBoostingRegressor(**gbr_params)
#
# Fit the model
#
gbr.fit(x_train_std, y_train)

In [ ]:
# Print Coefficient of determination R^2
#
print("Model Accuracy: %.3f" % gbr.score(x_test_std, y_test))
#
# Create the mean squared error
#
mse = mean_squared_error(y_test, gbr.predict(x_test_std))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
#
rmse= np.sqrt(mean_squared_error(y_test, x_test_std))

Smokers

In [ ]:
dffum = df[(df['smoker'] == 1)]
dffum

In [ ]:
X_fum = dffum.drop('charges',axis=1)
y_fum= dffum['charges']

In [ ]:
from sklearn.model_selection import train_test_split
X_train_fum, X_test_fum, y_train_fum, y_test_fum = train_test_split( X_fum, y_fum, test_size=0.2, random_state=42)
X_train_fum.shape, X_test_fum.shape

In [ ]:
clf_fum = LazyRegressor(verbose=0)
models_fum,predictions_fum = clf_fum.fit(X_train_fum, X_test_fum, y_train_fum, y_test_fum)
models_fum

Non Smokers

In [ ]:
dfnfum = df[(df['smoker'] == 0)]
dfnfum.sample(5)

In [ ]:
X_nfum = dfnfum.drop('charges',axis=1)
y_nfum= dfnfum['charges']

In [ ]:
from sklearn.model_selection import train_test_split
X_train_nfum, X_test_nfum, y_train_nfum, y_test_nfum = train_test_split( X_nfum, y_nfum, test_size=0.2, random_state=42)
X_train_nfum.shape, X_test_nfum.shape

In [ ]:
clf_nfum = LazyRegressor(verbose=0)
models_nfum,predictions_nfum = clf_nfum.fit(X_train_nfum, X_test_nfum, y_train_nfum, y_test_nfum)
models_nfum

Healthy BMI

In [ ]:
df_bmi_ok = df[(df['bmi']<=25)]
df_bmi_ok.sample(5)

In [ ]:
X_bmi_ok = df_bmi_ok.drop('charges',axis=1)
y_bmi_ok= df_bmi_ok['charges']

In [ ]:
from sklearn.model_selection import train_test_split
X_train_bmi_ok, X_test_bmi_ok, y_train_bmi_ok, y_test_bmi_ok = train_test_split( X_bmi_ok, y_bmi_ok, test_size=0.2, random_state=42)
X_train_bmi_ok.shape, X_test_bmi_ok.shape

In [ ]:
clf_bmi_ok = LazyRegressor(verbose=0)
models_bmi_ok,predictions_bmi_ok = clf_bmi_ok.fit(X_train_bmi_ok, X_test_bmi_ok, y_train_bmi_ok, y_test_bmi_ok)
models_bmi_ok

Unhealthy BMI

In [ ]:
df = pd.read_csv("../data/00-insurance.csv", index_col = 0).reset_index()
df.head(2)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['sex'] = df['sex'].map({'female':0,'male':1})
df['smoker'] = df['smoker'].map({'no':0,'yes':5})
df['region'] = df['region'].map({'northeast':3,'northwest':1,'southeast':4,'southwest':2})
df.head()

In [ ]:
df_bmi_nok = df[(df['bmi'] >25)]
df_bmi_nok.sample(5)

In [ ]:
X_bmi_nok = df_bmi_nok.drop('charges',axis=1)
y_bmi_nok= df_bmi_nok['charges']

In [ ]:
from sklearn.model_selection import train_test_split
X_train_bmi_nok, X_test_bmi_nok, y_train_bmi_nok, y_test_bmi_nok = train_test_split( X_bmi_nok, y_bmi_nok, test_size=0.2, random_state=42)
X_train_bmi_nok.shape, X_test_bmi_nok.shape

In [ ]:
clf_bmi_nok = LazyRegressor(verbose=0)
models_bmi_nok,predictions_bmi_nok = clf_bmi_nok.fit(X_train_bmi_nok, X_test_bmi_nok, y_train_bmi_nok, y_test_bmi_nok)
models_bmi_nok

In [ ]:
## Best metrics are given by GradientBooster, our response variable does not have a normal distribution, nor is the relationship with the response variables entirely linear. We could fit the model with interval prediction.

## Conclusions:

Our data includes mostly overweight, middle-aged clients with healthy smoking habits and no children. The trend lines of annual premiums are mostly conditioned by smoking and age (remembering that an unhealthy BMI is always the basis in most cases, and is an important driver of premiums).
Of these, smoking (difficult to monitor) and BMI (easily monitored) are variables.
Our model, is able to predict with accuracy: 0.826 and mean squared error (MSE) on test set: 31916978 the annual insurance premium with these variables.


## Proposal:

Creation of an interactive campaign in which potential new and existing customers can calculate the reduction in the amount of their annual insurance premium by enrolling in a weight reduction programme and maintaining their reduced BMI almost for a year (supervised) to achieve this reduction in the annual insurance premium, taking into account future ages for the calculation.

In [ ]:
1.Promote interaction of customers and prospective customers by calculating their insurance premiums and their decrease by improving their BMI
2.Facilitating access to slimming programmes for clients and future clients
3.Check annual monitoring of clients' maintenance of new BMI and healthy habits
4.Promoting healthy smoking habits (further benefits)

In [ ]:
Next steps

1.Develop a model to predict future spending after a year of declining BMI
2.Calculate benefit for the year during the insured's BMI decline while maintaining the old BMI premium.
3.Determine time in which health expenditure actually decreases with decreasing BMI